<a href="https://colab.research.google.com/github/110805/EEG_classification/blob/master/EEG_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import dataloader as loader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Hyperparameter setting
batch_size = 64
learning_rate = 1e-2
epochs = 300

# Loading data
train_data, train_label, test_data, test_label = loader.read_bci_data()

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()

        # firstconv
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1,51), stride=(1,1), padding=(0,25), bias=False)
        self.batchnorm1 = nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        # depthwiseconv
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(2,1), stride=(1,1), groups=16, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.avgpool1 = nn.AvgPool2d(kernel_size=(1,4), stride=(1,4), padding=0)

        # separableconv
        self.conv3 = nn.Conv2d(32, 32, kernel_size=(1,15), stride=(1,1), padding=(0,7), bias=False)
        self.batchnorm3 = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.avgpool2 = nn.AvgPool2d(kernel_size=(1,8), stride=(1,8), padding=0)

        # classify
        self.linear1 = nn.Linear(736, 2, bias=True)

    def forward(self, x):
        # firstconv
        out = self.conv1(x)
        out = self.batchnorm1(out)

        # depthwiseconv
        out = self.conv2(out)
        out = F.elu(self.batchnorm2(out), alpha=1.0)
        out = F.dropout(self.avgpool1(out), p=0.25)

        # separableconv
        out = self.conv3(out)
        out = F.elu(self.batchnorm3(out), alpha=1.0)
        out = F.dropout(self.avgpool2(out), p=0.25)

        # classify
        out = self.linear1(out)

        return out

model = EEGNet()
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


EEGNet(
  (conv1): Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False)
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False)
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (avgpool1): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (conv3): Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False)
  (batchnorm3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (avgpool2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
  (linear1): Linear(in_features=736, out_features=2, bias=True)
)